# 사람 추가

In [1]:
import os
import folium
import pandas as pd
import numpy as np
import networkx as nx

import seaborn as sns 
import matplotlib.pyplot as plt
plt.rc('font', family = 'NanumBarunGothic')
%matplotlib inline
from IPython.display import set_matplotlib_formats

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 사람/투석

In [3]:
df_node1 = pd.read_csv('/content/drive/Shareddrives/2022데이터마이닝응용/4 프로젝트/코드/종연_코드/투석_최종/투석_사람_nodes.csv', encoding = 'cp949')
df_link1 = pd.read_csv('/content/drive/Shareddrives/2022데이터마이닝응용/4 프로젝트/코드/종연_코드/투석_최종/투석_사람_links.csv', encoding = 'cp949')

In [7]:
links1 = df_link1[['Source', 'Target', 'Weight']]
links1.head()

,Source,Target,Weight
0,0.0,52.0,6.960529
1,0.0,58.0,4.729400
2,0.0,61.0,5.327075
3,0.0,62.0,6.434054
4,0.0,67.0,4.587192


In [ ]:
nodes1 = df_node1[['id', 'NAME', 'latitude', 'longitude','TYPE']]
nodes1.head()

,id,NAME,latitude,longitude,TYPE
0,0,환자,36.698989,127.481918,사람
1,1,환자,36.663737,127.417545,사람
2,2,환자,36.589344,127.487583,사람
3,3,환자,36.616492,127.519512,사람
4,4,환자,36.613333,127.540307,사람


### 네트워크 구성

In [ ]:
G1 = nx.Graph()
R = 6371e3

In [ ]:
for idx, row in nodes1.iterrows():
  G1.add_node(row['id'], Label=row['NAME'], latitude=row['latitude'], longitude=row['longitude'])

In [ ]:
for idx, row in links1.iterrows():
  G1.add_edge(row['Source'], row['Target'], weight = row['Weight'])

In [ ]:
print(nx.info(G1))

Graph with 96 nodes and 226 edges


### Folium

In [ ]:
std_point = tuple(nodes1.head(1)[['latitude', 'longitude']].iloc[0])
std_point

(36.6989887, 127.481918)

### 노드 표시

In [ ]:
map_osm = folium.Map(location=std_point, zoom_start=8)

for ix, row in nodes1.iterrows():
  location = (row['latitude'], row['longitude'])
  if row['NAME'] == '환자' :
    color = 'red'
  else :
    color = 'blue'
  folium.Circle(
      location = location,
      radius = 10*200,
      color = 'white',
      weight = 1,
      fill_opacity = 0.6,
      opacity = 1,
      fill_color = color ,
      fill = True 
  ).add_to(map_osm)

map_osm

In [ ]:
kw = {'opacity': 0.5, 'weight': 2}
for ix, row in links1.iterrows():
  start = tuple(nodes1[nodes1['id'] == row['Source']][['latitude', 'longitude']].iloc[0])
  end = tuple(nodes1[nodes1['id'] == row['Target']][['latitude', 'longitude']].iloc[0])
  folium.PolyLine(
      locations = [start, end],
      color = 'black',
      line_cap = 'round',
      **kw,
  ).add_to(map_osm)
map_osm

In [ ]:
#노드끼리 모두 연결 되어 있는지 확인한 결과 False(연결되어 있지 않음)로 확인되었다.
nx.is_connected(G1)

False

In [ ]:
#나뉘어 있는 네트워크의 개수를 확인한 결과로 140 개의 네트워크로 구성됨을 확인하였다.
nx.number_connected_components(G1)

25

In [ ]:
[c for c in sorted(nx.connected_components(G1), key=len, reverse=True)]

[{0,
  1.0,
  2.0,
  3.0,
  4.0,
  5.0,
  6.0,
  7.0,
  8.0,
  52.0,
  58.0,
  61.0,
  62.0,
  67.0,
  69.0,
  71.0,
  73.0,
  81.0,
  82.0,
  84.0,
  86.0,
  88.0,
  89.0,
  90.0,
  92.0,
  93.0,
  95.0},
 {9, 11.0, 12.0, 13.0, 14.0, 56.0, 60.0, 65.0, 70.0, 80.0, 83.0, 85.0, 87.0},
 {15, 16.0, 17.0, 19.0, 20.0, 53.0, 55.0, 63.0, 64.0, 74.0, 76.0},
 {21, 22.0, 23.0, 24.0, 91.0},
 {33, 34.0, 48.0, 77.0, 79.0},
 {25, 26.0, 27.0, 78.0},
 {28, 29.0, 30.0, 57.0},
 {39, 40.0, 66.0, 94.0},
 {31, 32.0, 59.0},
 {35, 36.0, 75.0},
 {38, 54.0},
 {49, 68.0},
 {10},
 {18},
 {37},
 {41},
 {42},
 {43},
 {44},
 {45},
 {46},
 {47},
 {50},
 {51},
 {72}]

In [ ]:
[nx.diameter(G1.subgraph(x)) for x in nx.connected_components(G1)]

[3, 3, 0, 4, 0, 2, 2, 2, 2, 2, 2, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]

In [ ]:
[nx.density(G1.subgraph(x)) for x in nx.connected_components(G1)]

[0.41595441595441596,
 0.3974358974358974,
 0,
 0.41818181818181815,
 0,
 0.4,
 0.5,
 0.5,
 0.6666666666666666,
 0.6,
 0.6666666666666666,
 0,
 1.0,
 0.6666666666666666,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1.0,
 0,
 0,
 0]

In [ ]:
[nx.transitivity(G1.subgraph(x)) for x in nx.connected_components(G1)]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
#print('diamter:', nx.diameter(G))
diameter = max([max(j.values()) for (i,j) in nx.shortest_path_length(G1)])
print(' Graph Stucture')
print(' diameter : ',diameter )
print(' density:', nx.density(G1))
print(' transitivity:', nx.transitivity(G1))

 Graph Stucture
 diameter :  4
 density: 0.04956140350877193
 transitivity: 0


# 사람/보건소/병원

In [9]:
df_node2 = pd.read_csv('/content/drive/Shareddrives/2022데이터마이닝응용/4 프로젝트/코드/종연_코드/투석_최종/투석_사람_보건소_nodes.csv', encoding = 'cp949')
df_link2 = pd.read_csv('/content/drive/Shareddrives/2022데이터마이닝응용/4 프로젝트/코드/종연_코드/투석_최종/투석_사람_보건소_links.csv', encoding = 'cp949')

In [15]:
df_node2.sample(frac=1).head(7)

,NAME,longitude,latitude,TYPE,id
108,청주시흥덕보건소,127.492857,36.640913,보건소,108
101,음성군보건소,127.688835,36.928825,보건소,101
19,환자,128.201733,37.129899,사람,19
94,신용봉내과의원,127.598395,36.989603,병원,94
223,괴산군연풍면내응보건진료소,127.981328,36.782434,보건소,223
258,단양군영춘면별방보건진료소,128.416596,37.094874,보건소,258
285,산대보건진료소,127.725251,36.573839,보건소,285


In [11]:
df_link2.head(7)

,LINK_ID,Source,Target,Weight
0,0.0,0.0,52.0,6.960529
1,1.0,0.0,58.0,4.729400
2,2.0,0.0,61.0,5.327075
3,3.0,0.0,62.0,6.434054
4,4.0,0.0,67.0,4.587192
5,5.0,0.0,69.0,6.654207
6,6.0,0.0,71.0,5.364705


In [16]:
links2 = df_link2[['Source', 'Target', 'Weight']]
links2.tail()

,Source,Target,Weight
1224,356.0,89.0,4.769601
1225,356.0,90.0,6.077417
1226,356.0,92.0,4.758572
1227,356.0,93.0,7.723483
1228,356.0,95.0,5.806746


In [17]:
nodes2 = df_node2[['id', 'NAME', 'latitude', 'longitude','TYPE']]
nodes2.head()
nodes2['TYPE'].value_counts()

보건소    268
사람      52
병원      44
Name: TYPE, dtype: int64

### 네트워크 구성

In [18]:
G2 = nx.Graph()
R = 6371e3

In [19]:
for idx, row in nodes2.iterrows():
  G2.add_node(row['id'], Label=row['NAME'], latitude=row['latitude'], longitude=row['longitude'])

In [20]:
for idx, row in links2.iterrows():
  G2.add_edge(row['Source'], row['Target'], weight = row['Weight'])

In [21]:
print(G2)

Graph with 364 nodes and 1229 edges


### Folium

In [22]:
std_point = tuple(nodes2.head(1)[['latitude', 'longitude']].iloc[0])
std_point

(36.6989887, 127.481918)

### 노드표시

In [23]:
map_osm = folium.Map(location=std_point, zoom_start=8)

for ix, row in nodes2.iterrows():
  location = (row['latitude'], row['longitude'])
  if row['TYPE'] == '사람' :
    color = 'red'
  elif row['TYPE'] == '병원':
    color = 'blue'
  else : 
    color = 'orange'
  folium.Circle(
      location = location,
      radius = 10*200,
      color = 'white',
      weight = 1,
      fill_opacity = 0.6,
      opacity = 1,
      fill_color = color ,
      fill = True 
  ).add_to(map_osm)

map_osm

In [24]:
kw = {'opacity': 0.5, 'weight': 2}
for ix, row in links2.iterrows():
  start = tuple(nodes2[nodes2['id'] == row['Source']][['latitude', 'longitude']].iloc[0])
  end = tuple(nodes2[nodes2['id'] == row['Target']][['latitude', 'longitude']].iloc[0])
  folium.PolyLine(
      locations = [start, end],
      color = 'black',
      line_cap = 'round',
      **kw,
  ).add_to(map_osm)
map_osm

In [ ]:
#노드끼리 모두 연결 되어 있는지 확인한 결과 False(연결되어 있지 않음)로 확인되었다.
nx.is_connected(G2)

False

In [ ]:
#나뉘어 있는 네트워크의 개수를 확인한 결과로 140 개의 네트워크로 구성됨을 확인하였다.
nx.number_connected_components(G2)

159

In [ ]:
[c for c in sorted(nx.connected_components(G2), key=len, reverse=True)]

In [ ]:
[nx.diameter(G2.subgraph(x)) for x in nx.connected_components(G2)]

In [ ]:
[nx.density(G2.subgraph(x)) for x in nx.connected_components(G2)]

In [ ]:
[nx.transitivity(G2.subgraph(x)) for x in nx.connected_components(G2)]

In [ ]:
#print('diamter:', nx.diameter(G))
diameter = max([max(j.values()) for (i,j) in nx.shortest_path_length(G2)])
print(' Graph Stucture')
print(' diameter : ',diameter )
print(' density:', nx.density(G2))
print(' transitivity:', nx.transitivity(G2))

In [ ]:
# 각 node별 centrality 값을 dictionary 형태로 보여줌
deg_cen = nx.degree_centrality(G2)
bet_cen = nx.betweenness_centrality(G2)
clo_cen = nx.closeness_centrality(G2)
eig_cen = nx.eigenvector_centrality(G2, max_iter=600) # max iter 의 deafault 값인 100으로 할 때 반복문이 끝나지 않아 600으로 다시 설정해줌
pagerank_cen = nx.pagerank(G2)



# 각 centrality별 가장 값이 높은 10개의 node 추출
total_list1 = []
deg_list = []
for node in deg_cen:
  if node in range(96, 364):
    deg_list.append((node, deg_cen[node]))

print('degree centrality:\n', sorted(deg_list, key=lambda x:x[1], reverse=True)[0:4])

for node in sorted(deg_list, key=lambda x:x[1], reverse=True)[0:10]:
  total_list1.append(node[0])

total_list2 = []
bet_list = []
for node in bet_cen:
  if node in range(96, 364):
    bet_list.append((node, bet_cen[node]))

print('betweenness centrality:\n', sorted(bet_list, key=lambda x:x[1], reverse=True)[0:4])

for node in sorted(bet_list, key=lambda x:x[1], reverse=True)[0:10]:
  total_list2.append(node[0])

total_list3 = []
clo_list = []
for node in clo_cen:
  if node in range(96, 364):
    clo_list.append((node, clo_cen[node]))
   
print('closeness centrality:\n', sorted(clo_list, key=lambda x:x[1], reverse=True)[0:4])

for node in sorted(clo_list, key=lambda x:x[1], reverse=True)[0:10]:
  total_list3.append(node[0])

total_list4 = []
eig_list = []
for node in eig_cen:
  if node in range(96, 364):
    eig_list.append((node, eig_cen[node]))

print('eigenvector centrality:\n', sorted(eig_list, key=lambda x:x[1], reverse=True)[0:4])

for node in sorted(eig_list, key=lambda x:x[1], reverse=True)[0:10]:
  total_list4.append(node[0])

total_list5 = []
pagerank_list = []
for node in pagerank_cen:
  if node in range(96, 364):
    pagerank_list.append((node, pagerank_cen[node]))

for node in sorted(pagerank_list, key=lambda x:x[1], reverse=True)[0:10]:
  total_list5.append(node[0])
   
print('page rank centrality:\n', sorted(pagerank_list, key=lambda x:x[1], reverse=True)[0:4])

degree centrality:
 [(106, 0.06611570247933884), (108, 0.06611570247933884), (142, 0.06611570247933884), (307, 0.06611570247933884)]
betweenness centrality:
 [(272, 0.06779268543823833), (340, 0.06570317525231922), (213, 0.054205492601436124), (229, 0.049323242344093264)]
closeness centrality:
 [(340, 0.15126050420168066), (142, 0.14555256064690025), (350, 0.14516129032258063), (354, 0.14516129032258063)]
eigenvector centrality:
 [(106, 0.1474058262960167), (108, 0.14555507238900017), (307, 0.14098127912639322), (356, 0.13527657638296856)]
page rank centrality:
 [(350, 0.007519561451285804), (142, 0.007395585694396156), (354, 0.007367407980016195), (351, 0.007199013543785341)]


In [ ]:
df_node2['NAME'][total_list1]

106           청주시서원보건소
108           청주시흥덕보건소
142            북이면보건지소
307           오근장보건진료소
356            탑리보건진료소
350    진천군 초평면 용산보건진료소
354            추학보건진료소
105           청주시상당보건소
107           청주시청원보건소
135             문백보건지소
Name: NAME, dtype: object

In [ ]:
df_node2['NAME'][total_list2]

272          매현보건진료소
340          주봉보건진료소
213          공전보건진료소
229    괴산군청천면신월보건진료소
204          회인면보건지소
298          신대보건진료소
101           음성군보건소
186       충주시대소원보건지소
143           불정보건지소
219    괴산군불정면추산보건진료소
Name: NAME, dtype: object

In [ ]:
df_node2['NAME'][total_list3]

340            주봉보건진료소
142            북이면보건지소
350    진천군 초평면 용산보건진료소
354            추학보건진료소
204            회인면보건지소
351      진천군문백면사양보건진료소
356            탑리보건진료소
101             음성군보건소
298            신대보건진료소
307           오근장보건진료소
Name: NAME, dtype: object

In [ ]:
df_node2['NAME'][total_list4]

106    청주시서원보건소
108    청주시흥덕보건소
307    오근장보건진료소
356     탑리보건진료소
105    청주시상당보건소
331     월오보건진료소
162     오창읍보건지소
332     유리보건진료소
142     북이면보건지소
117    남이면 보건지소
Name: NAME, dtype: object

In [ ]:
df_node2['NAME'][total_list5]

350    진천군 초평면 용산보건진료소
142            북이면보건지소
354            추학보건진료소
351      진천군문백면사양보건진료소
163            옥산면보건지소
135             문백보건지소
309            옥전보건진료소
204            회인면보건지소
186         충주시대소원보건지소
279            병암보건진료소
Name: NAME, dtype: object